In [1]:
import numpy as np
import cupy as cp
import numba 
import cupoch as cph
import time

In [2]:

class GetVoxel:
  def __init__(self, voxel_size):
    self.voxel_size = voxel_size
    self.pointcloud = cph.io.read_point_cloud("/home/dev/libs/cupoch/examples/testdata/fragment.pcd")
    # cph.visualization.draw_geometries([self.pointcloud])
    self.voxel_grid = cph.geometry.VoxelGrid.create_from_point_cloud(self.pointcloud, voxel_size=voxel_size)
    # cph.visualization.draw_geometries([voxel_grid])

    self.voxel_keys = list(self.voxel_grid.voxels.cpu().keys())
    print('voxel_keys_len', len(self.voxel_keys))

  def _create_pcd_from_voxel_keys(self):
        # Transfer data to GPU
    primitives_pos_gpu = cp.asarray(self.voxel_keys)
    offset = cp.asarray(self.voxel_grid.get_min_bound())
    voxel_size = cp.asarray(self.voxel_size)
    
    # Compute minimums for each column on GPU
    mins = cp.min(primitives_pos_gpu, axis=0)
    
    # Perform operations on GPU (Normalize and scale)
    # Subtract mins: Shifts all voxel indices so the minimum is at zero (origin).
    # Scale: Multiplies by the voxel size to convert from grid indices to real-world coordinates.
    # Offset: Adds the minimum bound and half a voxel size to center each primitive in its voxel.
    primitives_pos_gpu = primitives_pos_gpu - mins[None, :]
    primitives_pos_gpu = primitives_pos_gpu * voxel_size
    primitives_pos_gpu = primitives_pos_gpu + (offset + voxel_size/2)

    self.offset = offset
    self.mins = mins
    self.primitives_pos_gpu = primitives_pos_gpu
    primitives_pos = cp.asnumpy(primitives_pos_gpu)
    self.primitives_pos = primitives_pos
    return primitives_pos

  def create_pcd_from_voxel_keys(self):
    return self._create_pcd_from_voxel_keys()


voxel_size = 0.060
voxels = GetVoxel(voxel_size)
pcds = voxels.create_pcd_from_voxel_keys()

voxel_keys_len 1901


In [3]:
# cph.visualization.draw_geometries([cph.geometry.PointCloud(cph.utility.Vector3fVector(pcds))])

In [4]:
class SpatialHashingNN:
    def __init__(self, pcd, voxel_size):
        """
        Initialize the spatial hashing nearest neighbor structure.
        :param pcd: numpy array of point cloud (N, 3)
        :param voxel_size: float, the size of each voxel
        """
        self.pcd = pcd
        self.table_size = len(self.pcd) * 2
        self.voxel_size = voxel_size
        self.voxel_indices = {}
        self.hash_table = self.build_hash_table(pcd)

    def voxel_index(self, point):
        # Use floor for voxelization (common convention)
        x, y, z = np.floor(point / self.voxel_size)
        return (int(x), int(y), int(z))

    def hash_function(self, vi):
        p1 = 73856093
        p2 = 19349663
        p3 = 83492791
        # Standard spatial hash
        return (vi[0] * p1 ^ vi[1] * p2 ^ vi[2] * p3) % self.table_size

    def build_hash_table(self, points):
        hash_table = [-1] * self.table_size
        for idx, pt in enumerate(points):
            vi = self.voxel_index(pt)
            self.voxel_indices[idx] = vi
            hash_idx = self.hash_function(vi)
            while True:
                if hash_table[hash_idx] != -1:
                    # linear probing
                    hash_idx = (hash_idx + 1) % self.table_size
                else:
                    hash_table[hash_idx] = idx
                    break
        return hash_table

    def find_nn_for_point(self, idx):
        point = self.pcd[idx]
        vi = self.voxel_indices[idx]
        min_dist = float('inf')
        min_idx = -1

        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                for dz in [-1, 0, 1]:
                    nx = vi[0] + dx
                    ny = vi[1] + dy
                    nz = vi[2] + dz
                    hash_idx = self.hash_function((int(nx), int(ny), int(nz)))

                    attempts = 0
                    while attempts < self.table_size:
                        pt_idx = self.hash_table[hash_idx]
                        if pt_idx == -1:
                            break
                        nvi = self.voxel_indices[pt_idx]
                        if nvi[0] == nx and nvi[1] == ny and nvi[2] == nz and pt_idx != idx:
                            d2 = np.linalg.norm(point - self.pcd[pt_idx])
                            if d2 < min_dist:
                                min_dist = d2
                                min_idx = pt_idx
                            break
                        hash_idx = (hash_idx + 1) % self.table_size
                        attempts += 1
        return min_idx

    def find_all_nn(self):
        points = self.pcd
        nn_indices = []
        nn_distances = []
        for idx in range(len(points)):
            nn_idx = self.find_nn_for_point(idx)
            nn_indices.append(nn_idx)
            if nn_idx == -1:
                nn_distances.append(float('inf'))
            else:
                nn_dist = np.linalg.norm(points[idx] - points[nn_idx])
                nn_distances.append(nn_dist)
        return nn_indices, nn_distances
    
start = time.time()
shnn = SpatialHashingNN(pcds, voxel_size)
nn_indices, nn_distances = shnn.find_all_nn()
end = time.time()
print(f"Time taken: {end - start:.5f} seconds")

Time taken: 0.14908 seconds


In [5]:
class Evaluation:
  def __init__(self, pcd):
    self.pcd = pcd
    start = time.time()
    self._evaluate_nn_index_ground_truth()
    end = time.time()
    print(f"Time taken: {end - start:.5f} seconds")

  def _evaluate_nn_index_ground_truth(self):
    self.nn_index = []
    self.nn_index_dist = []
    for i in range(len(self.pcd)):
      min_dist = float('inf')
      min_dist_idx = None
      for j in range(len(self.pcd)):
        if i == j:
          continue
        dist = np.linalg.norm(self.pcd[i] - self.pcd[j])
        if dist < min_dist:
          min_dist = dist
          min_dist_idx = j
      self.nn_index.append(min_dist_idx)
      self.nn_index_dist.append(min_dist)
    
  def evaluate(self, nn_indices):
    hits = []
    for idx, nn_index in enumerate(nn_indices):
      if self.nn_index[idx] == nn_index:
        hits.append(True)
      else:
        hits.append(False)
    print(f"Accuracy: {(sum(hits) / len(hits)) * 100:.2f}%")
    return hits

eval = Evaluation(pcds)
hits = eval.evaluate(nn_indices)

Time taken: 10.09558 seconds
Accuracy: 81.01%


In [6]:
print(np.sum(eval.nn_index_dist), np.sum(nn_distances))

114.05999999999952 114.05999999999952


In [13]:
import numpy as np
from collections import defaultdict

class SpatialHashingNN:
    def __init__(self, pcd, voxel_size):
        """
        :param pcd: numpy array of shape (N, 3)
        :param voxel_size: float, size of each voxel
        """
        self.pcd = pcd
        self.voxel_size = voxel_size
        self.voxel_table = self.build_voxel_table(pcd)

    def voxel_index(self, point):
        # Use floor to assign negative coordinates correctly
        return tuple(np.floor(point / self.voxel_size).astype(int))

    def build_voxel_table(self, points):
        voxel_table = defaultdict(list)
        for idx, pt in enumerate(points):
            vi = self.voxel_index(pt)
            voxel_table[vi].append(idx)
        return voxel_table

    def find_nn_for_point(self, idx, max_radius=5):
        point = self.pcd[idx]
        vi = self.voxel_index(point)
        checked = set()
        min_dist = float('inf')
        min_idx = -1

        # Adaptive neighborhood expansion
        found_any = False
        for radius in range(1, max_radius+1):
            # Generate all voxel indices in a (2*radius+1)^3 cube
            for dx in range(-radius, radius+1):
                for dy in range(-radius, radius+1):
                    for dz in range(-radius, radius+1):
                        nvi = (vi[0]+dx, vi[1]+dy, vi[2]+dz)
                        if nvi in checked:
                            continue
                        checked.add(nvi)
                        for pt_idx in self.voxel_table.get(nvi, []):
                            if pt_idx == idx:
                                continue
                            d2 = np.linalg.norm(point - self.pcd[pt_idx])
                            if d2 < min_dist:
                                min_dist = d2
                                min_idx = pt_idx
                                found_any = True
            if found_any:
                break  # Stop expanding if at least one neighbor found
        return min_idx

    def find_all_nn(self):
        nn_indices = []
        nn_distances = []
        for idx in range(len(self.pcd)):
            nn_idx = self.find_nn_for_point(idx)
            nn_indices.append(nn_idx)
            if nn_idx == -1:
                nn_distances.append(float('inf'))
            else:
                nn_distances.append(np.linalg.norm(self.pcd[idx] - self.pcd[nn_idx]))
        return nn_indices, nn_distances

# Example usage:
pcds = np.random.rand(1000, 3)
voxel_size = 0.05
start = time.time()
shnn = SpatialHashingNN(pcds, voxel_size)
nn_indices, nn_distances = shnn.find_all_nn()
end = time.time()
print(f"Time taken: {end - start:.5f} seconds")

Time taken: 0.07574 seconds


In [10]:
nn_indices

[497,
 307,
 580,
 44,
 55,
 945,
 963,
 796,
 33,
 186,
 484,
 43,
 713,
 798,
 286,
 47,
 646,
 721,
 250,
 656,
 729,
 253,
 459,
 181,
 729,
 437,
 131,
 159,
 987,
 899,
 458,
 151,
 747,
 8,
 339,
 139,
 890,
 155,
 483,
 482,
 819,
 969,
 168,
 11,
 99,
 355,
 816,
 15,
 656,
 113,
 951,
 805,
 217,
 719,
 339,
 4,
 542,
 801,
 163,
 628,
 894,
 212,
 130,
 776,
 245,
 291,
 919,
 864,
 848,
 856,
 107,
 382,
 893,
 509,
 532,
 476,
 886,
 674,
 947,
 439,
 854,
 843,
 454,
 854,
 25,
 766,
 700,
 534,
 338,
 98,
 509,
 665,
 389,
 90,
 742,
 829,
 154,
 885,
 89,
 44,
 262,
 485,
 264,
 94,
 793,
 573,
 596,
 70,
 644,
 279,
 924,
 574,
 570,
 49,
 930,
 15,
 759,
 353,
 295,
 132,
 279,
 621,
 867,
 884,
 574,
 944,
 309,
 14,
 830,
 337,
 134,
 222,
 119,
 426,
 130,
 776,
 642,
 521,
 942,
 35,
 259,
 194,
 724,
 956,
 149,
 143,
 806,
 88,
 540,
 144,
 922,
 31,
 170,
 522,
 875,
 37,
 369,
 894,
 993,
 27,
 718,
 985,
 588,
 58,
 774,
 329,
 700,
 755,
 42,
 918,
 782,
 54